In [3]:
import streamlit as st
import pandas as pd
import joblib
import numpy as np
import os

# Set page configuration
st.set_page_config(
    page_title="Customer Churn Predictor",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Load the saved model and columns
@st.cache_resource
def load_model():
    try:
        model = joblib.load('churn_prediction_model.pkl')
        model_columns = joblib.load('model_columns.pkl')
        return model, model_columns
    except Exception as e:
        st.error(f"Model loading failed: {e}")
        return None, None

def create_input_data(form_data, model_columns):
    """Create input data for prediction"""
    input_data = {
        'tenure': form_data['tenure'],
        'MonthlyCharges': form_data['monthly_charges'],
        'TotalCharges': form_data['total_charges'],
        'is_new_customer': 1 if form_data['tenure'] <= 12 else 0,
        'is_long_term_customer': 1 if form_data['tenure'] > 24 else 0,
        'is_high_spender': 1 if form_data['monthly_charges'] > 70 else 0,
        'gender_Male': 1 if form_data['gender'] == "Male" else 0,
        'SeniorCitizen_Yes': 1 if form_data['senior_citizen'] == "Yes" else 0,
        'Partner_Yes': 1 if form_data['partner'] == "Yes" else 0,
        'Dependents_Yes': 1 if form_data['dependents'] == "Yes" else 0,
        'PhoneService_Yes': 1 if form_data['phone_service'] == "Yes" else 0,
        'MultipleLines_No phone service': 1 if form_data['multiple_lines'] == "No phone service" else 0,
        'MultipleLines_Yes': 1 if form_data['multiple_lines'] == "Yes" else 0,
        'InternetService_Fiber optic': 1 if form_data['internet_service'] == "Fiber optic" else 0,
        'InternetService_No': 1 if form_data['internet_service'] == "No" else 0,
        'OnlineSecurity_No internet service': 1 if form_data['online_security'] == "No internet service" else 0,
        'OnlineSecurity_Yes': 1 if form_data['online_security'] == "Yes" else 0,
        'OnlineBackup_No internet service': 1 if form_data['online_backup'] == "No internet service" else 0,
        'OnlineBackup_Yes': 1 if form_data['online_backup'] == "Yes" else 0,
        'DeviceProtection_No internet service': 1 if form_data['device_protection'] == "No internet service" else 0,
        'DeviceProtection_Yes': 1 if form_data['device_protection'] == "Yes" else 0,
        'TechSupport_No internet service': 1 if form_data['tech_support'] == "No internet service" else 0,
        'TechSupport_Yes': 1 if form_data['tech_support'] == "Yes" else 0,
        'StreamingTV_No internet service': 1 if form_data['streaming_tv'] == "No internet service" else 0,
        'StreamingTV_Yes': 1 if form_data['streaming_tv'] == "Yes" else 0,
        'StreamingMovies_No internet service': 1 if form_data['streaming_movies'] == "No internet service" else 0,
        'StreamingMovies_Yes': 1 if form_data['streaming_movies'] == "Yes" else 0,
        'Contract_One year': 1 if form_data['contract'] == "One year" else 0,
        'Contract_Two year': 1 if form_data['contract'] == "Two year" else 0,
        'PaperlessBilling_Yes': 1 if form_data['paperless_billing'] == "Yes" else 0,
        'PaymentMethod_Credit card (automatic)': 1 if form_data['payment_method'] == "Credit card (automatic)" else 0,
        'PaymentMethod_Electronic check': 1 if form_data['payment_method'] == "Electronic check" else 0,
        'PaymentMethod_Mailed check': 1 if form_data['payment_method'] == "Mailed check" else 0
    }
    
    # Create DataFrame with all model columns
    input_df = pd.DataFrame([input_data])
    
    # Ensure all model columns are present
    for col in model_columns:
        if col not in input_df.columns:
            input_df[col] = 0
    
    # Reorder columns to match training
    input_df = input_df[model_columns]
    
    return input_df

def display_results(prediction, prediction_proba, form_data):
    """Display prediction results"""
    # Results header
    st.header("PREDICTION RESULTS")
    st.markdown("---")
    
    # Main result cards
    col1, col2, col3 = st.columns([2,1,1])
    
    with col1:
        if prediction == 1:
            st.error("HIGH RISK: Customer Likely to CHURN")
            st.write("Immediate action required to retain this customer")
        else:
            st.success("LOW RISK: Customer Likely to STAY")
            st.write("Continue providing excellent service")
    
    with col2:
        st.metric(
            "Churn Probability", 
            f"{prediction_proba[1]*100:.1f}%"
        )
    
    with col3:
        risk_level = "High" if prediction_proba[1] > 0.7 else "Medium" if prediction_proba[1] > 0.4 else "Low"
        st.metric("Risk Level", risk_level)
    
    # Probability gauge
    st.subheader("RISK ASSESSMENT")
    st.progress(float(prediction_proba[1]))
    
    # Key factors analysis
    st.subheader("KEY RISK FACTORS")
    risk_factors = []
    
    if form_data['tenure'] <= 12:
        risk_factors.append("New customer (12 months or less tenure)")
    if form_data['monthly_charges'] > 70:
        risk_factors.append("High spender (over $70 monthly)")
    if form_data['contract'] == "Month-to-month":
        risk_factors.append("Month-to-month contract (no commitment)")
    if form_data['payment_method'] == "Electronic check":
        risk_factors.append("Electronic check payment method")
    if form_data['online_security'] == "No" and form_data['internet_service'] != "No":
        risk_factors.append("No online security service")
    
    if risk_factors:
        for factor in risk_factors:
            st.write(f"- {factor}")
    else:
        st.write("- No major risk factors identified")
    
    # Recommendations
    st.subheader("RECOMMENDATIONS")
    
    if prediction_proba[1] > 0.7:
        st.warning("HIGH RISK CUSTOMER - IMMEDIATE ACTION NEEDED")
        st.write("**Retention Strategies:**")
        st.write("- Offer loyalty discount or promotional rate")
        st.write("- Personal outreach from customer success team")
        st.write("- Review service quality and address issues")
        st.write("- Conduct satisfaction survey")
        st.write("- Provide exclusive benefits or premium trial")
        
    elif prediction_proba[1] > 0.4:
        st.info("MEDIUM RISK CUSTOMER - PROACTIVE MONITORING")
        st.write("**Engagement Strategies:**")
        st.write("- Regular check-ins every 2-3 months")
        st.write("- Ensure service satisfaction")
        st.write("- Consider proactive offers")
        st.write("- Gather feedback for improvement")
        st.write("- Monitor usage patterns")
        
    else:
        st.success("LOW RISK CUSTOMER - MAINTAIN RELATIONSHIP")
        st.write("**Growth Strategies:**")
        st.write("- Continue excellent service delivery")
        st.write("- Explore upselling opportunities")
        st.write("- Build long-term partnership")
        st.write("- Gather feedback for improvement")
        st.write("- Reward loyalty with benefits")

def main():
    # Header
    st.title("CUSTOMER CHURN PREDICTION APP")
    st.markdown("Predict customer churn risk using machine learning")
    st.markdown("---")
    
    # Load model
    model, model_columns = load_model()
    if model is None:
        st.error("Cannot load the machine learning model. Please ensure model files are available.")
        return
    
    # Create input form
    st.header("CUSTOMER INFORMATION")
    
    # Form in columns
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.subheader("Demographic & Billing")
        tenure = st.slider("Tenure (months)", 0, 72, 12)
        monthly_charges = st.number_input("Monthly Charges ($)", 0.0, 200.0, 70.0)
        total_charges = st.number_input("Total Charges ($)", 0.0, 10000.0, 1000.0)
        
    with col2:
        st.subheader("Personal Information")
        gender = st.selectbox("Gender", ["Female", "Male"])
        senior_citizen = st.selectbox("Senior Citizen", ["No", "Yes"])
        partner = st.selectbox("Partner", ["No", "Yes"])
        dependents = st.selectbox("Dependents", ["No", "Yes"])
        
    with col3:
        st.subheader("Service Information")
        internet_service = st.selectbox("Internet Service", ["DSL", "Fiber optic", "No"])
        contract = st.selectbox("Contract Type", ["Month-to-month", "One year", "Two year"])
        payment_method = st.selectbox("Payment Method", [
            "Bank transfer (automatic)", "Credit card (automatic)", 
            "Electronic check", "Mailed check"
        ])
        paperless_billing = st.selectbox("Paperless Billing", ["No", "Yes"])
    
    # Additional services
    st.subheader("Additional Services")
    col4, col5 = st.columns(2)
    
    with col4:
        st.write("**Communication Services**")
        phone_service = st.selectbox("Phone Service", ["No", "Yes"])
        multiple_lines = st.selectbox("Multiple Lines", ["No", "No phone service", "Yes"])
        
        st.write("**Security Services**")
        online_security = st.selectbox("Online Security", ["No", "No internet service", "Yes"])
        online_backup = st.selectbox("Online Backup", ["No", "No internet service", "Yes"])
        
    with col5:
        st.write("**Protection Services**")
        device_protection = st.selectbox("Device Protection", ["No", "No internet service", "Yes"])
        tech_support = st.selectbox("Tech Support", ["No", "No internet service", "Yes"])
        
        st.write("**Streaming Services**")
        streaming_tv = st.selectbox("Streaming TV", ["No", "No internet service", "Yes"])
        streaming_movies = st.selectbox("Streaming Movies", ["No", "No internet service", "Yes"])
    
    st.markdown("---")
    
    # Prediction button
    if st.button("PREDICT CHURN PROBABILITY", type="primary", use_container_width=True):
        # Collect form data
        form_data = {
            'tenure': tenure,
            'monthly_charges': monthly_charges,
            'total_charges': total_charges,
            'gender': gender,
            'senior_citizen': senior_citizen,
            'partner': partner,
            'dependents': dependents,
            'internet_service': internet_service,
            'contract': contract,
            'payment_method': payment_method,
            'paperless_billing': paperless_billing,
            'phone_service': phone_service,
            'multiple_lines': multiple_lines,
            'online_security': online_security,
            'online_backup': online_backup,
            'device_protection': device_protection,
            'tech_support': tech_support,
            'streaming_tv': streaming_tv,
            'streaming_movies': streaming_movies
        }
        
        with st.spinner("Analyzing customer data..."):
            # Create input data
            input_df = create_input_data(form_data, model_columns)
            
            # Make prediction
            prediction = model.predict(input_df)[0]
            prediction_proba = model.predict_proba(input_df)[0]
            
        # Display results
        display_results(prediction, prediction_proba, form_data)
    
    # Sidebar
    with st.sidebar:
        st.title("About This Project")
        st.markdown("""
        **AI/ML Customer Churn Prediction**
        
        This application predicts customer churn risk using machine learning based on:
        
        - Demographic data
        - Service usage patterns 
        - Billing information
        - Contract details
        
        **Model Performance:**
        - Accuracy: 79%
        - Precision: 82%
        - Recall: 89%
        
        **Key Insights:**
        - New customers have higher churn risk
        - High spenders are more likely to leave
        - Long-term contracts reduce churn risk
        
        Built with Streamlit & Scikit-learn
        """)
        
        st.markdown("---")
        st.markdown("**Developer:** AI/ML Portfolio Project")
        st.markdown("**Version:** 1.0")

if __name__ == "__main__":
    main()

2025-11-15 13:16:46.236 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 13:16:46.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 13:16:46.250 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 13:16:46.253 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 13:16:46.261 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 13:16:46.263 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 13:16:46.267 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 13:16:46.270 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [4]:
import os
print("Checking required files:")
required_files = ['app.py', 'churn_prediction_model.pkl', 'model_columns.pkl']
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file} - FOUND")
    else:
        print(f"❌ {file} - MISSING")

Checking required files:
✅ app.py - FOUND
✅ churn_prediction_model.pkl - FOUND
✅ model_columns.pkl - FOUND
